## Mixins

The following explores what implementing protocol suites using mixin classes could look like.

In theory, you could combine a secret-sharing scheme object with a set of scheme-agnostic arithmetic mixins to eliminate large swathes of boilerplate code.  In practice, I haven't figured-out a good way to handle the fact that different schemes manipulate different storage objects.  In C++ we might turn to traits objects to handle this.

In [1]:
from cicada.arithmetic import Field
from cicada.encoding import FixedPoint

##################################################################
# Secret sharing schemes

class AdditiveSharing(object):
    def __init__(self, encoding=None):
        if encoding is None:
            encoding = FixedPoint()
        self._encoding = encoding
    
    @property
    def encoding(self):
        return self._encoding
    
    def reshare(self, share):
        print("Resharing secret shares.")
    
    def reveal(self, share, dst=None, encoding=None):
        print("Revealing secret shares.")

    def share(self, src, secret, shape, encoding=None):
        print("Generating secret shares.")
    

##################################################################
# Scheme-specific field arithmetic.

class AdditiveFieldAdd(object):
    def field_add(self, lhs, rhs):
        print("Adding field values.")

class AdditiveFieldDivide(object):
    def field_divide(self, lhs, rhs):
        print("Dividing field values.")
        
class AdditiveFieldMultiplyDuAtallah(object):
    def field_multiply(self, lhs, rhs):
        print("Multiplying field values with Du and Atallah.")
    
class AdditiveFieldMultiplyBeaver(object):
    def field_multiply(self, lhs, rhs):
        print("Multiplying field values with Beaver triples.")

class AdditiveFieldSubtract(object):
    def field_subtract(self, lhs, rhs):
        print("Subtracting field values.")
        
class AdditiveRightShift(object):
    def right_shift(self, operand, bits):
        print(f"Shifting field values right {bits} bits.")

##################################################################
# Scheme-agnostic field arithmetic.

class FieldDot(object):
    def field_dot(self, lhs, rhs):
        print("Computing the dot product of field values.")
        result = self.field_multiply(lhs, rhs)
        result = self.sum(result)
        return result

class FieldEqual(object):
    def field_equal(self, lhs, rhs):
        print("Testing field values for equality.")

##################################################################
# Scheme-agnostic, encoding-specific arithmetic.

class Dot(object):
    def dot(self, lhs, rhs, encoding=None):
        if encoding is None:
            encoding = self.encoding
            
        print("Computing dot product.")
        result = self.field_dot(lhs, rhs)
        result = self.right_shift(result, encoding.precision)
        return result

class Multiply(object):
    def multiply(self, lhs, rhs, encoding=None):
        if encoding is None:
            encoding = self.encoding
            
        print("Multiplying real values.")
        result = self.field_multiply(lhs, rhs)
        result = self.right_shift(result, encoding.precision)
        return result
    
##################################################################
# Protocol suites.

class AdditiveProtocolSuite(Multiply, FieldEqual, FieldDot, AdditiveRightShift, AdditiveFieldMultiplyBeaver, AdditiveFieldAdd, AdditiveSharing):
    pass


In [2]:
protocol = AdditiveProtocolSuite()
a = protocol.share(src=0, secret=2.5, shape=())
b = protocol.share(src=1, secret=4, shape=())
c = protocol.multiply(a, b)
protocol.reveal(c)

Generating secret shares.
Generating secret shares.
Multiplying real values.
Multiplying field values with Beaver triples.
Shifting field values right 16 bits.
Revealing secret shares.
